In [ ]:
%load_ext autoreload 
%autoreload 2

In [ ]:
import json
from privacypacking.utils.utils import load_logs, global_metrics
import pandas as pd
from experiments.ray.analysis import load_ray_experiment, load_latest_ray_experiment, load_latest_scheduling_results, load_latest_scheduling_results, load_latest_ray_experiment
import plotly.express as px
from privacypacking.budget.curves import  LaplaceCurve, GaussianCurve, SubsampledGaussianCurve
from privacypacking.budget import Budget, Task, Block
from privacypacking.schedulers.metrics import OverflowRelevance, FlatRelevance
from privacypacking.budget.block_selection import RandomBlocks
from privacypacking.utils.plot import plot_budgets
import yaml
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from experiments.ray.analysis import get_percentiles

In [ ]:
# Budget totally unlocked from the beginning
# TODO: plot the delay. And before saving, add a line for Original DPF.
# rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-09_15-35-44"))

In [ ]:
# rdf.iloc[0].config

In [ ]:
# Older version with 100 mean tasks, no initial blocks, smaller profits.
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-09_15-35-44"))


# Unlocked, 500 mean tasks, running on C2 (mixed curves online.py)
# rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-10_03-35-42")) # incomplete (crashed)
# rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-10_13-46-19"))


px.line(
    rdf.query("T <= 10").sort_values("T"),
    x="T",
    y="realized_profit",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    # range_y=[0,1_000],
    # range_y=[0,25_000],
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
# rdf.query("T ==1").explode("allocated_tasks_scheduling_delays")
rdf["T"].unique()

In [ ]:
delays = rdf.query("T ==1").explode("allocated_tasks_scheduling_delays")
dpf = rdf.query("T == 0.01 and scheduler_metric == 'DominantShares'").explode("allocated_tasks_scheduling_delays")
dpf["scheduler_metric"] = "Original DPF"
delays = pd.concat([delays, dpf]).sort_values(["T", "scheduler_metric"])

px.ecdf(
    delays,
    x="allocated_tasks_scheduling_delays",
    color="scheduler_metric",

)

In [ ]:
delays.query("scheduler_metric == 'Fcfs'").allocated_tasks_scheduling_delays

In [ ]:
sq=delays.query(f"scheduler_metric == 'Original DPF'").allocated_tasks_scheduling_delays.value_counts()
series = sq.sort_index().cumsum()*1./len(sq)
indices = [0,100, 200, 300, 400, 500, 600]
d = pd.DataFrame(series).reset_index()
d["delay"] = d["index"]
d["fraction"] = d["allocated_tasks_scheduling_delays"]
d = d.drop(["index","allocated_tasks_scheduling_delays"], axis=1)
d

In [ ]:
sq=delays.query(f"scheduler_metric == 'Original DPF'").allocated_tasks_scheduling_delays.value_counts()
series = sq.sort_index().cumsum()*1./len(sq)
indices = [0,100, 200, 300, 400, 500, 600, 689]
d = pd.DataFrame(series).reset_index()
# d = pd.DataFrame(series).iloc[indices].reset_index()
d["delay"] = d["index"]
d["fraction"] = d["allocated_tasks_scheduling_delays"]
d = d.drop(["index","allocated_tasks_scheduling_delays"], axis=1)
d = d.append({"delay": 5, "fraction": 1},ignore_index=True)
d["scheduler_metric"] = "Original DPF"
cdf = d

indices = [0,100, 200, 300, 400, 500, 600, -1]
for m in delays.scheduler_metric.unique():
    if m != "Original DPF":
        sq=delays.query(f"scheduler_metric == '{m}'").allocated_tasks_scheduling_delays.value_counts()
        series = sq.sort_index().cumsum()*1./len(sq)
        d = pd.DataFrame(series).reset_index()
        d["delay"] = d["index"]
        d["fraction"] = d["allocated_tasks_scheduling_delays"]
        d = d.drop(["index","allocated_tasks_scheduling_delays"], axis=1)
        d = d.append({"delay": 5, "fraction": 1},ignore_index=True)
        d["scheduler_metric"] = m
        cdf = pd.concat([cdf, d])
cdf


In [ ]:
px.line(
    cdf,
    x="delay",
    y="fraction",
    color="scheduler_metric",
)

In [ ]:
def map_metric_to_id(row):
    d = {
        "DominantShares": 0,
        "DynamicFlatRelevance": 1,
        "BatchOverflowRelevance":2,
        "SoftKnapsack":3,
        "Original DPF":4,
    }
    return d[row]

gnuplot_df = cdf.query("scheduler_metric not in ['Fcfs', 'FlatRelevance']")
gnuplot_df["id"] = gnuplot_df.scheduler_metric.apply(map_metric_to_id)
gnuplot_df = gnuplot_df[["delay", "fraction", "id", "scheduler_metric"]].sort_values(["id","delay"])
gnuplot_df.to_csv("/home/pierre/privacypacking-paper/results/online_batched_mixed_curves/delay_cdf_unlocked_T1.csv", index=False)

In [ ]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-09_15-35-44"))
# rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-10_13-46-19"))

rdf.columns
rdf["avg_delay"] = rdf.allocated_tasks_scheduling_delays.apply(lambda x: np.mean(x))
rdf["p95"] = rdf.allocated_tasks_scheduling_delays.apply(lambda x: np.percentile(x, 95))

fig = px.line(
    rdf.sort_values(["T", "scheduler_metric"]).query("T <= 6"),
    x="T",
    y="avg_delay",
    color="scheduler_metric",
    log_x=True,
    width=1200,
    height=1000,
    # range_y=[0,15_000],
    title="10 + 20 blocks, privatekube, online. With and without profits.",
    facet_col="metric_recomputation_period",
    facet_row="data_path",
    facet_col_wrap=2,
)
fig

In [ ]:
# TODO: copy and add original DPF, then dump to gnuplot. No need actually?
def map_metric_to_id(row):
    d = {
        "DominantShares": 0,
        "DynamicFlatRelevance": 1,
        "BatchOverflowRelevance":2,
        "SoftKnapsack":3

    }
    return d[row]

gnuplot_df = rdf.query("T <= 10 and scheduler_metric not in ['Fcfs', 'FlatRelevance']")
gnuplot_df["id"] = gnuplot_df.scheduler_metric.apply(map_metric_to_id)
gnuplot_df = gnuplot_df[["T", "avg_delay", "id", "n_tasks", "scheduler", "scheduler_metric"]].sort_values(["id","T"])
gnuplot_df.to_csv("/home/pierre/privacypacking-paper/results/online_batched_mixed_curves/delay_online_batched_mixed_curves_N_without_profits_unlocked.csv", index=False)

In [ ]:
def map_metric_to_id(row):
    d = {
        "DominantShares": 0,
        "FlatRelevance": 1,
        "VectorizedBatchOverflowRelevance":2
    }
    return d[row]

gnuplot_df = rdf.query("T <= 10 and scheduler_metric not in ['Fcfs', 'DynamicFlatRelevance']")
gnuplot_df["id"] = gnuplot_df.scheduler_metric.apply(map_metric_to_id)
gnuplot_df = gnuplot_df[["T", "n_allocated_tasks", "id", "n_tasks", "scheduler", "scheduler_metric"]].sort_values(["id","T"])
gnuplot_df.to_csv("/home/pierre/privacypacking-paper/results/online_batched_mixed_curves/online_batched_mixed_curves_N_without_profits_unlocked.csv", index=False)

In [ ]:
gnuplot_df = rdf.query("T <= 10 and scheduler_metric != 'Fcfs'")

In [ ]:
def map_metric_to_id(m):
    d = {
        "DominantShares": 0,
        "FlatRelevance": 1,
        "VectorizedBatchOverflowRelevance":2
    }
    return d[m]

In [ ]:
gnuplot_df["id"] = gnuplot_df.scheduler_metric.apply(map_metric_to_id)

In [ ]:
gnuplot_df = gnuplot_df[["T", "n_allocated_tasks", "id", "n_tasks", "scheduler", "scheduler_metric"]].sort_values(["id","T"])

In [ ]:
gnuplot_df.to_csv("/home/pierre/privacypacking-paper/results/online_batched_mixed_curves/online_batched_mixed_curves_N_without_profits_unlocked.csv", index=False)

In [ ]:
px.line(
    percentile_df.query("T <= 10 and scheduler != 'Fcfs'"),
    x="T",
    y="delay",
    color="scheduler",
    line_dash="percentile",
    width=800,
    height=600,
    line_dash_map={50:"solid",25:"dash",99:"dash", 95:"dash"},
    title="Scheduling delay of allocated tasks, depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks<br> FCFS is 'batched FCFS'</sup>"
)

In [ ]:
def map_metric_to_id(row):
    d = {
        ("VectorizedBatchOverflowRelevance", 95):0,
        ("FlatRelevance", 95): 1,
        ("DominantShares", 95): 2,
        ("VectorizedBatchOverflowRelevance", 50):3,
        ("FlatRelevance", 50): 4,
        ("DominantShares", 50): 5,
    }
    return d[(row.scheduler, row.percentile)]

gnuplot_df = percentile_df.query("T <= 10 and scheduler not in ['Fcfs', 'DynamicFlatRelevance']")
gnuplot_df["id"] = gnuplot_df.apply(map_metric_to_id, axis=1)
gnuplot_df = gnuplot_df[["T", "delay", "id", "scheduler", "percentile"]].sort_values(["id","T"])
gnuplot_df.to_csv("/home/pierre/privacypacking-paper/results/online_batched_mixed_curves/delay_online_batched_mixed_curves_N_without_profits_unlocked.csv", index=False)

In [ ]:
# Gradual unlocking
# Old version
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-09_18-51-49"))
# Running on C2 at 22:41

# Initial blocks, profits and 500 mean tasks
# rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-10_03-39-52")) # incomplete
# rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-10_13-47-44"))
# Just not so great in the end. What is the point of unlocking in this case?
px.line(
    rdf.sort_values("T"),
    x="T",
    y="realized_profit",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    range_x=[0.1, 10],
    range_y=[0,1_000],
    # range_y=[0,25_000],
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
delays = rdf.query("T ==1").explode("allocated_tasks_scheduling_delays")
dpf = rdf.query("T == 0.01 and scheduler_metric == 'DominantShares'").explode("allocated_tasks_scheduling_delays")
dpf["scheduler_metric"] = "Original DPF"
delays = pd.concat([delays, dpf]).sort_values(["T", "scheduler_metric"])

px.ecdf(
    delays,
    x="allocated_tasks_scheduling_delays",
    color="scheduler_metric",

)

In [ ]:
sq=delays.query(f"scheduler_metric == 'Original DPF'").allocated_tasks_scheduling_delays.value_counts()
series = sq.sort_index().cumsum()*1./len(sq)
indices = [0,100, 200, 300, 400, 500, 600, 689]
d = pd.DataFrame(series).reset_index()
# d = pd.DataFrame(series).iloc[indices].reset_index()
d["delay"] = d["index"]
d["fraction"] = d["allocated_tasks_scheduling_delays"]
d = d.drop(["index","allocated_tasks_scheduling_delays"], axis=1)
d = d.append({"delay": 5, "fraction": 1},ignore_index=True)
d["scheduler_metric"] = "Original DPF"
cdf = d

indices = [0,100, 200, 300, 400, 500, 600, -1]
for m in delays.scheduler_metric.unique():
    if m != "Original DPF":
        sq=delays.query(f"scheduler_metric == '{m}'").allocated_tasks_scheduling_delays.value_counts()
        series = sq.sort_index().cumsum()*1./len(sq)
        d = pd.DataFrame(series).reset_index()
        d["delay"] = d["index"]
        d["fraction"] = d["allocated_tasks_scheduling_delays"]
        d = d.drop(["index","allocated_tasks_scheduling_delays"], axis=1)
        d = d.append({"delay": 5, "fraction": 1},ignore_index=True)
        d["scheduler_metric"] = m
        cdf = pd.concat([cdf, d])
cdf

In [ ]:
px.line(
    cdf,
    x="delay",
    y="fraction",
    color="scheduler_metric",
)

In [ ]:
def map_metric_to_id(row):
    d = {
        "DominantShares": 0,
        "DynamicFlatRelevance": 1,
        "BatchOverflowRelevance":2,
        "SoftKnapsack":3,
        "Original DPF":4,
    }
    return d[row]

gnuplot_df = cdf.query("scheduler_metric not in ['Fcfs', 'FlatRelevance']")
gnuplot_df["id"] = gnuplot_df.scheduler_metric.apply(map_metric_to_id)
gnuplot_df = gnuplot_df[["delay", "fraction", "id", "scheduler_metric"]].sort_values(["id","delay"])
gnuplot_df.to_csv("/home/pierre/privacypacking-paper/results/online_batched_mixed_curves/delay_cdf_locked_T1.csv", index=False)

In [ ]:
# TODO: copy and add original DPF, then dump to gnuplot. No need actually?
def map_metric_to_id(row):
    d = {
        "DominantShares": 0,
        "DynamicFlatRelevance": 1,
        "BatchOverflowRelevance":2,
        "SoftKnapsack":3

    }
    return d[row]

gnuplot_df = rdf.query("T <= 10 and scheduler_metric not in ['Fcfs', 'FlatRelevance']")
gnuplot_df["id"] = gnuplot_df.scheduler_metric.apply(map_metric_to_id)
gnuplot_df = gnuplot_df[["T", "n_allocated_tasks", "id", "n_tasks", "scheduler", "scheduler_metric"]].sort_values(["id","T"])
gnuplot_df.to_csv("/home/pierre/privacypacking-paper/results/online_batched_mixed_curves/online_batched_mixed_curves_N_without_profits_locked.csv", index=False)

In [ ]:
# Gradual unlocking.
# Diff seed (1)
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-09_20-55-13"))
px.line(
    rdf.sort_values("T"),
    x="T",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1000],
    title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
# Delay
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-09_18-51-49"))


# rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-09_20-55-13"))
# rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-10_13-46-19"))

rdf.columns
rdf["avg_delay"] = rdf.allocated_tasks_scheduling_delays.apply(lambda x: np.mean(x))
rdf["p95"] = rdf.allocated_tasks_scheduling_delays.apply(lambda x: np.percentile(x, 95))

fig = px.line(
    rdf.sort_values(["T", "scheduler_metric"]),
    x="T",
    y="avg_delay",
    color="scheduler_metric",
    # log_x=True,
    width=1200,
    height=1000,
    # range_y=[0,15_000],
    title="10 + 20 blocks, privatekube, online. With and without profits.",
    facet_col="metric_recomputation_period",
    facet_row="data_path",
    facet_col_wrap=2,
)
fig

In [ ]:
# TODO: copy and add original DPF, then dump to gnuplot. No need actually?
def map_metric_to_id(row):
    d = {
        "DominantShares": 0,
        "DynamicFlatRelevance": 1,
        "BatchOverflowRelevance":2,
        "SoftKnapsack":3

    }
    return d[row]

gnuplot_df = rdf.query("T <= 10 and scheduler_metric not in ['Fcfs', 'FlatRelevance']")
gnuplot_df["id"] = gnuplot_df.scheduler_metric.apply(map_metric_to_id)
gnuplot_df = gnuplot_df[["T", "avg_delay", "id", "n_tasks", "scheduler", "scheduler_metric"]].sort_values(["id","T"])
gnuplot_df.to_csv("/home/pierre/privacypacking-paper/results/online_batched_mixed_curves/delay_online_batched_mixed_curves_N_without_profits_locked.csv", index=False)

In [ ]:
# Gradual unlocking.
# Diff seed (1) and 10 initial blocks.
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-09_20-55-13"))
px.line(
    rdf.sort_values("T"),
    x="T",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1000],
    title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
def map_metric_to_id(row):
    d = {
        "DominantShares": 0,
        "FlatRelevance": 1,
        "VectorizedBatchOverflowRelevance":2
    }
    return d[row]

gnuplot_df = rdf.query("T <= 10 and scheduler_metric not in ['Fcfs', 'DynamicFlatRelevance']")
gnuplot_df["id"] = gnuplot_df.scheduler_metric.apply(map_metric_to_id)
gnuplot_df = gnuplot_df[["T", "n_allocated_tasks", "id", "n_tasks", "scheduler", "scheduler_metric"]].sort_values(["id","T"])
gnuplot_df.to_csv("/home/pierre/privacypacking-paper/results/online_batched_mixed_curves/online_batched_mixed_curves_N_without_profits_locked.csv", index=False)

In [ ]:
px.line(
    percentile_df.query("T <= 10 and scheduler != 'Fcfs'"),
    x="T",
    y="delay",
    color="scheduler",
    line_dash="percentile",
    width=800,
    height=600,
    line_dash_map={50:"solid",25:"dash",99:"dash", 95:"dash"},
    title="Scheduling delay of allocated tasks, depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks<br> FCFS is 'batched FCFS'</sup>"
)

In [ ]:
def map_metric_to_id(row):
    d = {
        ("VectorizedBatchOverflowRelevance", 95):0,
        ("FlatRelevance", 95): 1,
        ("DominantShares", 95): 2,
        ("VectorizedBatchOverflowRelevance", 50):3,
        ("FlatRelevance", 50): 4,
        ("DominantShares", 50): 5,
    }
    return d[(row.scheduler, row.percentile)]

gnuplot_df = percentile_df.query("T <= 10 and scheduler not in ['Fcfs', 'DynamicFlatRelevance']")
gnuplot_df["id"] = gnuplot_df.apply(map_metric_to_id, axis=1)
gnuplot_df = gnuplot_df[["T", "delay", "id", "scheduler", "percentile"]].sort_values(["id","T"])
gnuplot_df.to_csv("/home/pierre/privacypacking-paper/results/online_batched_mixed_curves/delay_online_batched_mixed_curves_N_without_profits_locked.csv", index=False)